# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv', index_col = False)
cities_df

,Unnamed: 0,City ID,City,Country,Lng,Lat,Cloudiness,Date,Humidity,Max Temp,Wind Speed
0,0,2069194,Jamestown,AU,138.60,-33.21,14.0,"Friday, 15. November 2019 11:36AM",95.0,46.238,2.12
1,1,5106834,Albany,US,-73.75,42.65,20.0,"Friday, 15. November 2019 11:35AM",45.0,45.266,4.60
2,2,1039536,Maxixe,MZ,35.35,-23.86,40.0,"Friday, 15. November 2019 11:36AM",65.0,80.870,4.10
3,3,3652764,Puerto Ayora,EC,-90.35,-0.74,75.0,"Friday, 15. November 2019 11:36AM",65.0,77.270,7.20
4,4,3366880,Hermanus,ZA,19.24,-34.42,2.0,"Friday, 15. November 2019 11:36AM",79.0,72.266,1.79
5,5,5332698,Calexico,US,-115.50,32.68,1.0,"Friday, 15. November 2019 11:31AM",48.0,70.268,1.50
6,6,4020109,Atuona,PF,-139.03,-9.80,6.0,"Friday, 15. November 2019 11:36AM",72.0,80.474,6.13
7,7,2075265,Busselton,AU,115.35,-33.64,38.0,"Friday, 15. November 2019 11:36AM",28.0,70.268,7.89
8,8,6138501,Saint-Augustin,CA,-73.98,45.63,90.0,"Friday, 15. November 2019 11:35AM",64.0,38.264,6.20
9,9,3471451,Arraial do Cabo,BR,-42.02,-22.97,75.0,"Friday, 15. November 2019 11:36AM",88.0,71.870,9.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity_weight = cities_df["Humidity"].astype(float)


In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
address =  ['Hermanus', 'Mexicali', 'Hilo']
hotel_df = cities_df[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)& (cities_df['Wind Speed'] < 10)& (cities_df['Cloudiness'] < 3)]


hotel_df
# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

,Unnamed: 0,City ID,City,Country,Lng,Lat,Cloudiness,Date,Humidity,Max Temp,Wind Speed
4,4,3366880,Hermanus,ZA,19.24,-34.42,2.0,"Friday, 15. November 2019 11:36AM",79.0,72.266,1.79
5,5,5332698,Calexico,US,-115.50,32.68,1.0,"Friday, 15. November 2019 11:31AM",48.0,70.268,1.50
14,14,5855927,Hilo,US,-155.08,19.71,1.0,"Friday, 15. November 2019 11:33AM",64.0,70.268,2.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# geocoordinates
target_coordinates = ['-34.42, 19.24','32.68, -115.50','19.71,-155.08']
# target_search = "ZA"
target_radius = 5000
target_type = "hotels"

# set up a parameters dictionary
params = {
    "location": target_coordinates[1],
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [8]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "CqQCHQEAAP5kKPh6vQKJjsZ4o8XRx6pQD8NfZ94FdturuCsBdmgVOFLhVpcJw9Phk44oAIWCczcF8iZ8_EIR7xyOZMI8_l_ysOFpTtTzt98CKe5T2A-xLXolKQ4bOlUmw7nRzc-yctJYeqaAKmbRh6PJYdF4Rqav2w67JXOigvjAbo6pb-xhh-UbRbZszljx5dLvn1Farl7NBkA5RY_rCBeEGbF4Icx6cTGpnIogDda-hG6g8Nken5DnkGSRBjtPkV8AFhw92pdiIKddMPkgmMvr0fUXs9uCw6-UmmeiL7xOC99jWrJ4wkKnMkYSftXOda5XvIrUzWak0tbokVEaDdw5wUIdzB8lGJ_GJosc8OPWeIts3soT1hq24DIX9ANxxYv48CjuqxIQK6DvqtyVLr7xMOgXZXgbiBoUV5o7c5FnRdIFPEK4lwmI9Wag8no",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.62453889999999,
                    "lng": -115.4522623
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.6723463,
                        "lng": -115.3417969
                    },
                    "southwest": {
                        "lat": 32.5551708,
                        "lng": -115.5587851
          

In [9]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Mexicali
Mexicali


In [10]:

narrowed_city_df = hotel_df.assign(**{'Hotel Name' : address })
 
narrowed_city_df


,Unnamed: 0,City ID,City,Country,Lng,Lat,Cloudiness,Date,Humidity,Max Temp,Wind Speed,Hotel Name
4,4,3366880,Hermanus,ZA,19.24,-34.42,2.0,"Friday, 15. November 2019 11:36AM",79.0,72.266,1.79,Hermanus
5,5,5332698,Calexico,US,-115.50,32.68,1.0,"Friday, 15. November 2019 11:31AM",48.0,70.268,1.50,Mexicali
14,14,5855927,Hilo,US,-155.08,19.71,1.0,"Friday, 15. November 2019 11:33AM",64.0,70.268,2.60,Hilo


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

# Display Map

In [13]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer([(-34.42, 19.24),(32.68, -115.50),(19.71,-155.08)])
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))